# 📘 Project Summary
This notebook loads employee data from a JSON file and saves it into a Neo4j graph database using py2neo.

## 💡 What this means:
We are turning our data into nodes and relationships, like drawing a network of people and their skills.

## ✅ What we did:
- Created one node for each department (like "Administration").
- Added a function to create employee nodes from any department.
- Linked each employee to their department with a connection.
- For each employee, we:
- Added the skills they know.
- Stored how good they are at each skill (like 85%) as part of the connection.
- Even if some employee details are missing (like email), we still add empty values so all employees have the same type of info.

In [2]:
import json
from py2neo import Graph, Node, Relationship, NodeMatcher

In [3]:
#Make Connection
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

In [4]:
# Load employee data from the JSON file
with open("data/employees.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [5]:
top_keys = list(data.keys())
print("Top Level Keys: ", top_keys)

Top Level Keys:  ['Employee']


In [6]:
top_key = top_keys[0]

print("Departments: ", data[top_key].keys())

Departments:  dict_keys(['Administration', 'Unknown Department', 'Technician Team', 'Drivers', 'Customer Support Team', 'Human Resource Department'])


In [7]:
# Get all department names
departments = data["Employee"].keys()

# Create a node for each department
for dept_name in departments:
    # Check if the department node already exists to avoid duplicates
    existing_node = graph.nodes.match("Department", name=dept_name).first()
    if not existing_node:
        dept_node = Node("Department", name=dept_name)
        graph.create(dept_node)
        print(f"Created Department node: {dept_name}")
    else:
        print(f"Department node already exists: {dept_name}")

Created Department node: Administration
Created Department node: Unknown Department
Created Department node: Technician Team
Created Department node: Drivers
Created Department node: Customer Support Team
Created Department node: Human Resource Department


![Departments](data/pics/departments.png)

## Add one employee of administration department

In [8]:
def create_employee(dept_name, employee_index):
    employees = data["Employee"].get(dept_name)
    if employees is None:
        print(f"❌ Department '{dept_name}' not found.")
        return

    if employee_index >= len(employees):
        print(f"❌ Index {employee_index} out of range for department '{dept_name}'.")
        return

    emp = employees[employee_index]
    skill_tags = emp.get("skill_tags", [])
    skills_list = emp.get("skills", [])

    # Create or find Employee
    emp_node = graph.nodes.match("Employee", id=emp.get("id")).first()
    if not emp_node:
        emp_node = Node(
            "Employee",
            id=emp.get("id"),
            name=emp.get("name") or "",
            gender=emp.get("gender") or "",
            contact=emp.get("contact") or "",
            email=emp.get("email") or "",
            private_address=emp.get("private_address") or "",
            job_title=emp.get("job_title") or "",
            working_shift=emp.get("working_shift") or "",
            skill_tags=skill_tags
        )
        graph.create(emp_node)
        print(f"✅ Created Employee: {emp.get('name')}")
    else:
        print(f"⚠️ Employee already exists: {emp.get('name')}")
        return

    # Link to Department
    dept_node = graph.nodes.match("Department", name=dept_name).first()
    if dept_node:
        graph.merge(Relationship(emp_node, "WORKS_IN", dept_node))
    else:
        print(f"❌ Department node not found: {dept_name}")

    # Create Skill nodes & relationships
    for skill in skills_list:
        skill_name = skill.get("name")
        progress = skill.get("progress")

        if skill_name:
            skill_node = graph.nodes.match("Skill", name=skill_name).first()
            if not skill_node:
                skill_node = Node("Skill", name=skill_name)
                graph.create(skill_node)

            rel = Relationship(emp_node, "HAS_SKILL", skill_node, progress=progress)
            graph.merge(rel)

In [9]:
create_employee("Administration", 0)

✅ Created Employee: MMK


![Employee](data/pics/add_emp1.png)

## Add Employee of Technician Team

In [10]:
create_employee("Technician Team", 3)

✅ Created Employee: Manelyn NoynaySanchez


![Employee](data/pics/add_emp2.png)

In [11]:
create_employee("Human Resource Department", 0)

✅ Created Employee: Pranali Ramkrishna Kadam


## Create all employee of a particular department

In [12]:
def create_all_employees(dept_name):
    employees = data["Employee"].get(dept_name)
    
    if not employees:
        print(f"❌ Department '{dept_name}' not found or has no employees.")
        return
    
    for idx in range(len(employees)):
        create_employee(dept_name, idx)

In [13]:
create_all_employees("Administration")

⚠️ Employee already exists: MMK
✅ Created Employee: Generalist


In [14]:
create_all_employees("Unknown Department")

✅ Created Employee: Peter
✅ Created Employee: Mir
✅ Created Employee: Mashusa
✅ Created Employee: Tehir


![Unk Employee](data/pics/unk_dept.png)

In [15]:
create_all_employees("Drivers")

✅ Created Employee: Safeer Ahmad Shah Muhammad
✅ Created Employee: Muhammad Asad Asghar Ali
✅ Created Employee: Waheed Ahmed Nisar Ahmed
✅ Created Employee: Ali Raza Muhammad Mehdi
✅ Created Employee: Muhammad Usman Khushi
✅ Created Employee: Zain Ul Hadi Noor Ul Hadi
✅ Created Employee: Nadeem
✅ Created Employee: Abu Hurraira
✅ Created Employee: Abdul Razzaq
✅ Created Employee: Adnan Yousaf
✅ Created Employee: Muhammad Asif
✅ Created Employee: Muhammad Nabeel
✅ Created Employee: Muhammad Shahid Muhammad Khalid
✅ Created Employee: Zeeshan Khalid Mehmod


In [16]:
create_all_employees("Customer Support Team")

✅ Created Employee: Nikytasha Bhatia
✅ Created Employee: Omosivie Encharia Emokpaire
✅ Created Employee: Chloe Cameleon Buhawe Ruiz
✅ Created Employee: Mary Grace Bautista Jubinal
✅ Created Employee: Najat Mohamed Faraj
✅ Created Employee: Shahul Hameed
✅ Created Employee: Ashley Ann Belderol


![Department](data/pics/add_dept.png)

In [17]:
create_employee("Technician Team", 3)

⚠️ Employee already exists: Manelyn NoynaySanchez


![Department](data/pics/add_dept2.png)

### Note that the skill percentage is store in relationship

In [18]:
create_all_employees("Technician Team")

✅ Created Employee: MMK
✅ Created Employee: test employee
✅ Created Employee: Ethel Makwah
⚠️ Employee already exists: Manelyn NoynaySanchez
✅ Created Employee: Mary Dela Cruz Talusig
✅ Created Employee: Marife Genotivo
✅ Created Employee: Edie Enone
✅ Created Employee: Madhusa Thirimana
✅ Created Employee: Margarita Cuarema
✅ Created Employee: Ceril Cresenciana Drilon Cabalin
✅ Created Employee: Mirna Ayuni Paramida
✅ Created Employee: NI MADE Ayu Suarmini
✅ Created Employee: Ni Gusti Ayu Purnamawati
✅ Created Employee: Mellisa Mojares
✅ Created Employee: Melody Maique Villatuya
✅ Created Employee: Ni Komang Ayu Astini
✅ Created Employee: Ni MD kiki Indrawati
✅ Created Employee: Ni Putu Ekayanti
✅ Created Employee: Padmi Endrowati
✅ Created Employee: Rahela
✅ Created Employee: Gillian Nkom Andinwo
✅ Created Employee: Puspa Lata Phurumbu
✅ Created Employee: Dhana Maya Awasthi
✅ Created Employee: Esteph Zacarias
✅ Created Employee: Victoria Robert
✅ Created Employee: Rollyn Quitor Salva

![Department](data/pics/all_dept.png)